<a href="https://colab.research.google.com/github/2303A51758/MITS-Internship/blob/main/medium_level_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1: RESTful Services for Internship Applications & Certificate Verification
This app includes:

Internship application submission API

Certificate verification API

Input validation using Pydantic

SQLite database via SQLAlchemy

Simple architecture for fast testing & development

1. Install Required Packages



In [1]:
pip install fastapi uvicorn sqlalchemy pydantic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00


2. Full Python Code (Single File: main.py)

In [4]:
!pip install fastapi uvicorn sqlalchemy "pydantic[email]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.5 MB/s eta 0:00:00


In [5]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, EmailStr
from sqlalchemy import create_engine, Column, Integer, String, Date
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import date
import uvicorn

# ================= Database Setup =================

DATABASE_URL = "sqlite:///./internship.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autoflush=False)
Base = declarative_base()

# ================= Models =================

class InternshipApplication(Base):
    __tablename__ = "applications"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, nullable=False)
    email = Column(String, nullable=False)
    domain = Column(String, nullable=False)
    application_date = Column(Date, default=date.today)

class Certificate(Base):
    __tablename__ = "certificates"
    certificate_code = Column(String, primary_key=True, index=True)
    recipient_name = Column(String, nullable=False)
    issue_date = Column(Date, nullable=False)

Base.metadata.create_all(bind=engine)

# ================= Schemas =================

class ApplicationRequest(BaseModel):
    name: str
    email: EmailStr
    domain: str

class ApplicationResponse(ApplicationRequest):
    id: int
    application_date: date
    class Config:
        orm_mode = True

class CertificateResponse(BaseModel):
    certificate_code: str
    recipient_name: str
    issue_date: date
    class Config:
        orm_mode = True

# ================= FastAPI App =================

app = FastAPI()
ALLOWED_DOMAINS = ["AI", "Cybersecurity", "Web Development", "Data Science"]

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# ================= API Routes =================

@app.post("/applications", response_model=ApplicationResponse)
def submit_application(app_data: ApplicationRequest):
    if app_data.domain not in ALLOWED_DOMAINS:
        raise HTTPException(status_code=400, detail="Invalid domain")
    db = next(get_db())
    application = InternshipApplication(
        name=app_data.name,
        email=app_data.email,
        domain=app_data.domain
    )
    db.add(application)
    db.commit()
    db.refresh(application)
    return application

@app.get("/certificates/{code}", response_model=CertificateResponse)
def verify_certificate(code: str):
    db = next(get_db())
    cert = db.query(Certificate).filter(Certificate.certificate_code == code).first()
    if not cert:
        raise HTTPException(status_code=404, detail="Certificate not found")
    return cert

# ================= Run Server =================

if __name__ == "__main__":
    uvicorn.run("main:app", port=8000, reload=True)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [1418] using StatReload
INFO:     Stopping reloader process [1418]


3. How to Use
▶️ Start the Server

In [9]:
!python main.py

python3: can't open file '/content/main.py': [Errno 2] No such file or directory


Access API Docs:
Open your browser and visit:




http://127.0.0.1:8000/docs


4. Example API Usage
📤 POST /applications

In [15]:
{
  "name": "John Doe",
  "email": "john@example.com",
  "domain": "AI"
}


{'name': 'John Doe', 'email': 'john@example.com', 'domain': 'AI'}

5. (Optional) Insert Test Certificate Manually

In [17]:
# Save the FastAPI application code into main.py
%%writefile main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, EmailStr
from sqlalchemy import create_engine, Column, Integer, String, Date
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import date
import uvicorn

# ================= Database Setup =================

DATABASE_URL = "sqlite:///./internship.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autoflush=False)
Base = declarative_base()

# ================= Models =================

class InternshipApplication(Base):
    __tablename__ = "applications"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, nullable=False)
    email = Column(String, nullable=False)
    domain = Column(String, nullable=False)
    application_date = Column(Date, default=date.today)

class Certificate(Base):
    __tablename__ = "certificates"
    certificate_code = Column(String, primary_key=True, index=True)
    recipient_name = Column(String, nullable=False)
    issue_date = Column(Date, nullable=False)

Base.metadata.create_all(bind=engine)

# ================= Schemas =================

class ApplicationRequest(BaseModel):
    name: str
    email: EmailStr
    domain: str

class ApplicationResponse(ApplicationRequest):
    id: int
    application_date: date
    class Config:
        orm_mode = True

class CertificateResponse(BaseModel):
    certificate_code: str
    recipient_name: str
    issue_date: date
    class Config:
        orm_mode = True

# ================= FastAPI App =================

app = FastAPI()
ALLOWED_DOMAINS = ["AI", "Cybersecurity", "Web Development", "Data Science"]

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# ================= API Routes =================

@app.post("/applications", response_model=ApplicationResponse)
def submit_application(app_data: ApplicationRequest):
    if app_data.domain not in ALLOWED_DOMAINS:
        raise HTTPException(status_code=400, detail="Invalid domain")
    db = next(get_db())
    application = InternshipApplication(
        name=app_data.name,
        email=app_data.email,
        domain=app_data.domain
    )
    db.add(application)
    db.commit()
    db.refresh(application)
    return application

@app.get("/certificates/{code}", response_model=CertificateResponse)
def verify_certificate(code: str):
    db = next(get_db())
    cert = db.query(Certificate).filter(Certificate.certificate_code == code).first()
    if not cert:
        raise HTTPException(status_code=404, detail="Certificate not found")
    return cert

# ================= Run Server =================

if __name__ == "__main__":
    uvicorn.run("main:app", port=8000, reload=True)

Writing main.py
